In [1]:
from sklearn.model_selection import RandomizedSearchCV

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import pandas as pd

In [2]:
ls

Homemade CRF.ipynb  build/              notebooks/          scripts/
MANIFEST.in         dist/               parsing_tools/      setup.py
Makefile            greenr/             raw_data/           tests/
README.md           greenr.egg-info/    requirements.txt


In [35]:
training_data = pd.read_csv('tests/golden/training_data.crf', sep = "\t")
testing_data = pd.read_csv('tests/golden/testing_data.crf', sep = "\t")

In [36]:
training_data.columns = ['name', 'position', 'L', 'cap', 'paren', 'tag' ]

In [37]:
testing_data.columns = ['name', 'position', 'L', 'cap', 'paren', 'tag' ]

In [38]:
training_data 

,name,position,L,cap,paren,tag
0,cups,I2,L20,NoCAP,NoPAREN,B-UNIT
1,cooked,I3,L20,NoCAP,NoPAREN,B-COMMENT
2,and,I4,L20,NoCAP,NoPAREN,I-COMMENT
3,pureed,I5,L20,NoCAP,NoPAREN,I-COMMENT
4,fresh,I6,L20,NoCAP,NoPAREN,I-COMMENT
...,...,...,...,...,...,...
115463,note,I23,LX,NoCAP,YesPAREN,I-COMMENT
115464,),I24,LX,NoCAP,YesPAREN,OTHER
115465,Pinch,I1,L4,YesCAP,NoPAREN,OTHER
115466,dried,I2,L4,NoCAP,NoPAREN,B-COMMENT


In [39]:
testing_data

,name,position,L,cap,paren,tag
0,freshly,I2,L8,NoCAP,NoPAREN,B-COMMENT
1,ground,I3,L8,NoCAP,NoPAREN,I-COMMENT
2,black,I4,L8,NoCAP,NoPAREN,B-NAME
3,pepper,I5,L8,NoCAP,NoPAREN,I-NAME
4,1,I1,L8,NoCAP,NoPAREN,B-QTY
...,...,...,...,...,...,...
13313,1,I1,L8,NoCAP,NoPAREN,B-QTY
13314,teaspoon,I2,L8,NoCAP,NoPAREN,B-UNIT
13315,finely,I3,L8,NoCAP,NoPAREN,B-COMMENT
13316,minced,I4,L8,NoCAP,NoPAREN,I-COMMENT


In [40]:
index = training_data[training_data['position']=='I1'].index

In [41]:
test_index = testing_data[testing_data['position']=='I1'].index

In [42]:
recipes = []
for i in range(len(index)):

    recipes.append(training_data.iloc[index[i-1]:index[i]].set_index('name').to_records())

In [43]:
test_recipes = []
for i in range(len(test_index)):

        test_recipes.append(testing_data.iloc[test_index[i-1]:test_index[i]].set_index('name').to_records())


In [44]:
test_recipes[1]

rec.array([('1', 'I1', 'L8', 'NoCAP', 'NoPAREN', 'B-QTY'),
           ('tablespoon', 'I2', 'L8', 'NoCAP', 'NoPAREN', 'B-UNIT'),
           ('medium-hot', 'I3', 'L8', 'NoCAP', 'NoPAREN', 'B-COMMENT'),
           ('paprika', 'I4', 'L8', 'NoCAP', 'NoPAREN', 'B-NAME')],
          dtype=[('name', 'O'), ('position', 'O'), ('L', 'O'), ('cap', 'O'), ('paren', 'O'), ('tag', 'O')])

In [45]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, position, postag, cap, paren, label in sent]

def sent2tokens(sent):
    return [token for token, position, postag, cap, paren, label in sent]

In [15]:
sent2features(recipes[2])[0]

{'bias': 1.0,
 'word.lower()': '1',
 'word[-3:]': '1',
 'word[-2:]': '1',
 'word.isupper()': False,
 'word.istitle()': False,
 'word.isdigit()': True,
 'postag': 'I1',
 'postag[:2]': 'I1',
 'BOS': True,
 '+1:word.lower()': 'medium-size',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'I2',
 '+1:postag[:2]': 'I2'}

In [73]:
recipes[2]

rec.array([('1', 'I1', 'L8', 'NoCAP', 'NoPAREN', 'B-QTY'),
           ('medium-size', 'I2', 'L8', 'NoCAP', 'NoPAREN', 'B-COMMENT'),
           ('onion', 'I3', 'L8', 'NoCAP', 'NoPAREN', 'B-NAME'),
           (',', 'I4', 'L8', 'NoCAP', 'NoPAREN', 'B-COMMENT'),
           ('peeled', 'I5', 'L8', 'NoCAP', 'NoPAREN', 'I-COMMENT'),
           ('and', 'I6', 'L8', 'NoCAP', 'NoPAREN', 'I-COMMENT'),
           ('chopped', 'I7', 'L8', 'NoCAP', 'NoPAREN', 'I-COMMENT')],
          dtype=[('name', 'O'), ('position', 'O'), ('L', 'O'), ('cap', 'O'), ('paren', 'O'), ('tag', 'O')])

In [46]:
%%time
X_train = [sent2features(recipe) for recipe in recipes[1:]]
y_train = [sent2labels(recipe) for recipe in recipes[1:]]

X_test = [sent2features(recipe) for recipe in test_recipes[1:]]
y_test = [sent2labels(recipe) for recipe in test_recipes[1:]]

CPU times: user 6.88 s, sys: 95.3 ms, total: 6.98 s
Wall time: 6.99 s


In [32]:
test_recipes[1]

IndexError: list index out of range

In [47]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 14.8 s, sys: 109 ms, total: 14.9 s
Wall time: 15 s


CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [20]:
labels = list(crf.classes_)
labels

['B-QTY',
 'B-UNIT',
 'I-COMMENT',
 'B-NAME',
 'B-COMMENT',
 'OTHER',
 'I-NAME',
 'B-RANGE_END']

In [48]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.8524973170763069

In [30]:
X_test[0]

IndexError: list index out of range

In [49]:
crf

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [50]:
y_pred

[['B-QTY', 'B-UNIT', 'B-NAME', 'I-NAME'],
 ['B-QTY',
  'B-UNIT',
  'B-COMMENT',
  'B-NAME',
  'B-COMMENT',
  'I-COMMENT',
  'I-COMMENT',
  'I-COMMENT',
  'B-NAME',
  'I-COMMENT'],
 ['B-QTY', 'B-UNIT', 'B-NAME', 'OTHER', 'B-COMMENT', 'I-COMMENT', 'I-COMMENT'],
 ['B-QTY', 'B-UNIT', 'B-NAME', 'I-NAME', 'I-NAME'],
 ['B-QTY',
  'B-UNIT',
  'B-NAME',
  'I-NAME',
  'OTHER',
  'B-COMMENT',
  'I-COMMENT',
  'OTHER'],
 ['B-QTY',
  'B-UNIT',
  'B-COMMENT',
  'B-NAME',
  'I-NAME',
  'B-COMMENT',
  'I-COMMENT',
  'I-COMMENT',
  'I-COMMENT',
  'OTHER',
  'I-COMMENT',
  'I-COMMENT',
  'I-COMMENT',
  'I-COMMENT',
  'I-COMMENT'],
 ['B-QTY',
  'B-NAME',
  'I-NAME',
  'B-COMMENT',
  'I-COMMENT',
  'I-COMMENT',
  'I-COMMENT',
  'I-COMMENT',
  'I-COMMENT',
  'I-COMMENT'],
 ['B-QTY', 'B-UNIT', 'B-NAME', 'I-NAME'],
 ['B-QTY', 'B-NAME', 'OTHER', 'B-COMMENT', 'I-COMMENT', 'I-COMMENT'],
 ['B-QTY',
  'B-UNIT',
  'B-COMMENT',
  'B-NAME',
  'I-NAME',
  'B-COMMENT',
  'I-COMMENT',
  'I-COMMENT',
  'I-COMMENT'],
 ['

In [51]:
def get_ingredients_url(url):

    page = requests.get(f'{url}')
    soup = BeautifulSoup(page.content, 'html.parser')
    ingredient = []

    for a in soup.find_all('li', class_ = "recipe-ingredients__list-item"):
        ingredient.append(a.get_text())


    return ingredient

In [56]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
import re
import subprocess
import json
import pandas as pd
import numpy as np
test_ing = get_ingredients_url('https://www.bbc.co.uk/food/recipes/hummus_chickpea_burgers_94137')

In [57]:
test_ing

['3-4 tbsp sunflower oil',
 '1 onion, roughly chopped',
 '2 garlic cloves, thinly sliced',
 '1 tsp ground cumin',
 '1 tsp ground coriander',
 '400g tin chickpeas, drained and rinsed',
 '100g/3½oz hummus',
 '50g/2oz plain flour',
 '½ tsp salt',
 'freshly ground black pepper',
 '50g/2oz pine nuts or cashews',
 '8 pitta breads, warmed',
 'mixed green salad',
 '8 tbsp plain yoghurt (optional)',
 '8 tsp chilli sauce']

In [71]:
from nltk.tokenize import word_tokenize

crf.predict(word_tokenize(test_ing[4])[2])

[['B-QTY'], ['B-QTY'], ['B-QTY'], ['B-QTY'], ['B-QTY'], ['B-QTY']]

[{'bias': 1.0,
  'word.lower()': '1',
  'word[-3:]': '1',
  'word[-2:]': '1',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': True,
  'postag': 'I1',
  'postag[:2]': 'I1',
  'BOS': True,
  '+1:word.lower()': 'tablespoon',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'I2',
  '+1:postag[:2]': 'I2'},
 {'bias': 1.0,
  'word.lower()': 'tablespoon',
  'word[-3:]': 'oon',
  'word[-2:]': 'on',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'I2',
  'postag[:2]': 'I2',
  '-1:word.lower()': '1',
  '-1:word.istitle()': False,
  '-1:word.isupper()': False,
  '-1:postag': 'I1',
  '-1:postag[:2]': 'I1',
  '+1:word.lower()': 'medium-hot',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'I3',
  '+1:postag[:2]': 'I3'},
 {'bias': 1.0,
  'word.lower()': 'medium-hot',
  'word[-3:]': 'hot',
  'word[-2:]': 'ot',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit(

In [80]:
def unclump(s):
    """
    Replacess $'s with spaces. The reverse of clumpFractions.
    """
    return re.sub(r'\$', " ", s)


In [81]:
unclump('1$5e cup water')

'1 5e cup water'

In [97]:
def getFeatures(token, index, tokens):
    """
    Returns a list of features for a given token.

    """
    length = len(tokens)

    return [("I%s" % index), ("L%s" % lengthGroup(length)),
            ("Yes" if isCapitalized(token) else "No") + "CAP",
            ("Yes" if insideParenthesis(token, tokens) else "No") + "PAREN"]

In [107]:
ls

__init__.py                  ghg_calculator.ipynb
__pycache__/                 ingredient_phrase_tagger/
app.py                       lib.py
background.png               main_calculation.py
calculator.py                matching.py
data/                        scraper_parser.py
df_recorded_similarities.pk  style.css
df_wiki_similarities.pk      vectorizer.pk


In [120]:
import re
import json
from ingredient_phrase_tagger.training import utils
from string import punctuation

from nltk.tokenize import *

tokenizer = PunktSentenceTokenizer()

def get_sentence_features(sent):
    """Gets  the features of the sentence"""
    sent_tokens = nltk.word_tokenize(utils.cleanUnicodeFractions(sent))

    sent_features = []
    for i, token in enumerate(sent_tokens):
        token_features = [token]
        token_features.extend(utils.getFeatures(token, i+1, sent_tokens))
        sent_features.append(token_features)
    return sent_features



In [126]:
crf.predict(get_sentence_features('2 cups water'))

[['I-NAME', 'I-NAME', 'I-NAME', 'I-NAME', 'I-NAME'],
 ['I-NAME', 'I-NAME', 'I-NAME', 'I-NAME', 'I-NAME'],
 ['I-NAME', 'I-NAME', 'I-NAME', 'I-NAME', 'I-NAME']]

In [122]:
get_sentence_features('2 cups water')[0].to

['2', 'I1', 'L4', 'NoCAP', 'NoPAREN']